Installing transformers library

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.7MB/s 
     |████████████████████████████████| 1.1MB 15.5MB/s 
     |████████████████████████████████| 3.0MB 20.7MB/s 
     |████████████████████████████████| 890kB 31.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0e160f0d8e330cddfae288d670e7ef09658cbc06e2c11a5542f50639262f29ef
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Importing dependencies

In [ ]:
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torchvision import transforms
from transformers import AutoTokenizer, AutoModel

Network class defines the architecture and forward propagation of the model used to fine tune the pretrained work embeddings model, and generate meaningful sentence embeddings.

In [ ]:
class Network(nn.Module):
  
  def __init__(self, input_size, hidden_size, num_layers):
    
    super(Network, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True)
    self.fc = nn.Linear(hidden_size*2, hidden_size*2)
        
  def forward_once(self, x, length):

    x = nn.utils.rnn.pack_padded_sequence(x, length, batch_first=True)
    x, (hidden, cell) = self.lstm(x)
    x = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
    x = self.fc(x)
    return x
    
  def forward(self, input1, length1, input2, length2):

    output1 = self.forward_once(input1, length1)
    output2 = self.forward_once(input2, length2)
    return output1, output2

Defines the contrastive loss function. The loss function used to train the similarity model.

In [ ]:
class ContrastiveLoss(torch.nn.Module):
  
  def __init__(self, margin=2.0):

    super(ContrastiveLoss, self).__init__()
    self.margin = margin

  def forward(self, output1, output2, label):
    
    euclidean_distance = F.pairwise_distance(output1, output2)
    loss_contrastive = torch.mean((label) * torch.pow(euclidean_distance, 2) + (1-label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
    return loss_contrastive

Checking for availability of GPU.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Loading pretrained word embeddings model.

Link to pretrained model: https://huggingface.co/gsarti/biobert-nli?text=The+goal+of+life+is+%5BMASK%5D.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
model = AutoModel.from_pretrained("gsarti/biobert-nli",output_hidden_states=True)
model.to(device)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

Loading the training and test set.

In [ ]:
df_chunk = pd.read_csv("TrainingSet.csv", encoding="cp1252")#Replace path with path to training set csv file
df_chunk_test = pd.read_csv("TestSet.csv", encoding="cp1252")#Replace path with path to test set csv file

Utility function that returns the contents of the columns of training/test set data in separate lists. Takes as input a pandas variable.

In [ ]:
def get_data(df):
  
  anchor=[]
  positive=[]
  label=[]
  for index,row in df.iterrows():
    anchor.append(row['Question1'])
    positive.append(row['Question2'])
    label.append(row['Label'])
  return anchor, positive, label

Utility function that returns the a list of word embeddings corresponding to the input list of sentences(strings).

In [ ]:
def get_chunk_embedding(list_input):
  
  em_cls=[]
  for question in list_input:
    ids=tokenizer.encode(question)
    ids = torch.LongTensor(ids).unsqueeze(0)
    em_cls.append(ids)
  sentence_embeddings=[]
  with torch.no_grad():
    for x in em_cls:
      x=x.to(device)
      sentence_embeddings.append((model(input_ids=x)[2])[-1])
  return sentence_embeddings

Functions that returns list of word embeddings for Question1 and Question2 (first 2 columns of training/test set data) and a list of labels(3rd column of training/test set data). Takes as input a pandas variable.

In [ ]:
def get_embedding(df):

  df.columns=["Question1","Question2","Label"]
  anchor, positive, labels = get_data(df)
  s_ea = get_chunk_embedding(anchor)
  s_ep = get_chunk_embedding(positive)
  return s_ea, s_ep, labels

Loading the word embeddings for Question1 and Question2 (first 2 columns of training set data) and a list of labels(3rd column of training set data), if they have been previously calculated and saved.

In [ ]:
sentence_embeddingsa = torch.load("drive/My Drive/sa.pt")#Replace with path to file containing word embeddings for Question1(first column of training set data)
sentence_embeddingsp = torch.load("drive/My Drive/sp.pt")#Replace with path to file containing word embeddings for Question2(second column of training set data)
labels = torch.load("drive/My Drive/l.pt")#Replace with path to file containing labels(third column of training set data)

Calling the get_embedding function for training set data.

In [ ]:
sentence_embeddingsa, sentence_embeddingsp, labels = get_embedding(df_chunk)

                                               Question1  ... Label
0      What is the frequency of deviated septum, and ...  ...     1
1      Is there a hemoglobin A1C (glycosylated hemogl...  ...     1
2      How does pregnancy affect serum cholesterol le...  ...     1
3      I've got a woman with stage I breast cancer, 2...  ...     0
4      What is the drug of choice to start with in Ty...  ...     1
...                                                  ...  ...   ...
10481   What is the dose of cholestyramine for diarrhea?  ...     0
10482  Can a flare up of her pancreatitis cause relat...  ...     1
10483  Are her headaches related to her elevated dias...  ...     0
10484  How meaningful are these fluctuations in chole...  ...     0
10485  Is the meningococcal vaccine recommended for c...  ...     1

[10486 rows x 3 columns]


Saving the word embeddings for Question1 and Question2 (first 2 columns of training set data) and a list of labels(3rd column of training set data), if they have been previously calculated.

In [ ]:
torch.save(sentence_embeddingsa, "drive/My Drive/sa.pt")#Replace destination path to save file containing word embeddings for Question1(first column of training set data)
torch.save(sentence_embeddingsp, "drive/My Drive/sp.pt")#Replace destination path to save file containing word embeddings for Question2(second column of training set data)
torch.save(labels, "drive/My Drive/l.pt")#Replace destination path to save file containing labels(third column of training set data)

Loading the word embeddings for Question1 and Question2 (first 2 columns of test set data) and a list of labels(3rd column of test set data), if they have been previously calculated and saved.

In [ ]:
tsentence_embeddingsa = torch.load("drive/My Drive/tsa.pt")#Replace with path to file containing word embeddings for Question1(first column of test set data)
tsentence_embeddingsp = torch.load("drive/My Drive/tsp.pt")#Replace with path to file containing word embeddings for Question2(second column of test set data)
tlabels = torch.load("drive/My Drive/tl.pt")#Replace with path to file containing labels(third column of test set data)

Calling the get_embedding function for test set data.

In [ ]:
tsentence_embeddingsa, tsentence_embeddingsp, tlabels = get_embedding(df_chunk_test)

                                              Question1  ... Label
0             What would cause a solitary rectal ulcer?  ...     0
1     What is the difference between restless legs s...  ...     1
2     What agent should I use next in this hypertens...  ...     1
3     What are the interactions of the new antidepre...  ...     0
4        What does the prenatal care checklist include?  ...     0
...                                                 ...  ...   ...
2075  Patient has a prolapsed uterus and is having p...  ...     1
2076  Why does the prenatal amniocentesis questionna...  ...     1
2077  Patient complains she has been dizzy and can't...  ...     0
2078  A large-for-gestational-age baby was delivered...  ...     0
2079  Ideal treatment for peripheral edema from nons...  ...     1

[2080 rows x 3 columns]


Saving the word embeddings for Question1 and Question2 (first 2 columns of test set data) and a list of labels(3rd column of test set data), if they have been previously calculated.

In [ ]:
torch.save(tsentence_embeddingsa, "drive/My Drive/tsa.pt")#Replace with destination path to save word embeddings for Question1(first column of test set data)
torch.save(tsentence_embeddingsp, "drive/My Drive/tsp.pt")#Replace with destination path to save word embeddings for Question2(second column of test set data)
torch.save(tlabels, "drive/My Drive/tl.pt")#Replace with destination path to save labels(third column of test set data)

Defining the parameters for the Network class and training of the similarity model.

In [ ]:
input_size = 768
sequence_length = 512
num_layers = 2
hidden_size = 768
learning_rate = 0.001
epochs = 50

Creating a new instance of Network class and setting the loss criterion and optimizer.

In [ ]:
SimilarityModel = Network(input_size, hidden_size, num_layers)
SimilarityModel = SimilarityModel.to(device)
criterion = ContrastiveLoss()
optimizer = optim.Adam(SimilarityModel.parameters(), lr = learning_rate)
SimilarityModel.train()

Network(
  (lstm): LSTM(768, 768, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1536, out_features=1536, bias=True)
)

The training loop for the similarity model, forward propagates through the model calculates loss and carries out backpropagation.

In [ ]:
for epoch in tqdm(range(epochs), desc="Epochs"):
    running_loss = []

    for a,p,l in zip(sentence_embeddingsa,sentence_embeddingsp,labels):
      
      a = a.to(device)
      la = torch.tensor([a.shape[1]])
      p = p.to(device)
      lp = torch.tensor([p.shape[1]])

      optimizer.zero_grad()
      a, p = SimilarityModel(a, la, p, lp)

      loss = criterion(a , p, l)
      loss.backward()
      optimizer.step()
      running_loss.append(loss.cpu().detach().numpy())

    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(running_loss)))

KeyboardInterrupt: ignored

Code to save the trained similarity model.

In [ ]:
torch.save(nnmodel.state_dict(),"drive/My Drive/SimilarityModel.pt")#Replace with destination path to save the trained model

Loading the trained model if already saved.

In [ ]:
SimilarityModel = Network(input_size, hidden_size, num_layers)
SimilarityModel.load_state_dict(torch.load("drive/My Drive/SimilarityModel.pt"))#Replace with path to the saved trained model
SimilarityModel = SimilarityModel.to(device)

Calculating accuracy on the training set.

In [ ]:
SimilarityModel.eval()
total=0
total1=0
total0=0
for s,p,l in zip(sentence_embeddingsa,sentence_embeddingsp,labels):
  s = s.to(device)
  p = p.to(device)
  x, y=SimilarityModel(s,torch.tensor([s.shape[1]]),p,torch.tensor([p.shape[1]]))
  x=x.to(device)
  y=y.to(device)
  out = F.pairwise_distance(x, y)
  if out<=0.7 and l==1:
    total+=1
    total1+=1
  if out>0.7 and l==0:
    total+=1
    total0+=1
print("Training set->")
print("Total accuracy: ", total/len(labels))
print("Accuracy on true labels: ", total1/labels.count(1))
print("Accuracy on false labels: ", total0/labels.count(0))

Training set->
Total accuracy:  0.9673850848750716
Accuracy on true labels:  0.9604212055192447
Accuracy on false labels:  0.9750903977501004


Calculating accuracy on the test set.

In [ ]:
SimilarityModel.eval()
total=0
total1=0
total0=0
for s,p,l in zip(tsentence_embeddingsa,tsentence_embeddingsp,tlabels):
  s = s.to(device)
  p = p.to(device)
  x, y=SimilarityModel(s,torch.tensor([s.shape[1]]),p,torch.tensor([p.shape[1]]))
  x=x.to(device)
  y=y.to(device)
  out = F.pairwise_distance(x, y)
  if out<=0.7 and l==1:
    total+=1
    total1+=1
  if out>0.7 and l==0:
    total+=1
    total0+=1
print("Test set->")
print("Total accuracy: ", total/len(tlabels))
print("Accuracy on true labels: ", total1/tlabels.count(1))
print("Accuracy on false labels: ", total0/tlabels.count(0))

Test set->
Total accuracy:  0.9692307692307692
Accuracy on true labels:  0.9594594594594594
Accuracy on false labels:  0.9804123711340206
